# Import Required Libraries
Import necessary libraries such as NumPy, VTK, and vmtk.

In [26]:
# Import Required Libraries
import numpy as np
import vtk
import vmtk
from vmtk.vmtkscripts import vmtkCenterlines
from vmtk.vmtkscripts import vmtkPointListSeedSelector

# Define Helper Functions
Define all the helper functions from the provided code, including `point_cloud_to_watertight_mesh`, `mesh_to_volume`, `visualize_mesh_with_centerline`, `slice_mesh_and_connect_centroids`, and others.

In [32]:
# Helper Functions

def point_cloud_to_watertight_mesh(point_cloud_file, output_mesh_file, alpha=0.1):
    """Converts a point cloud (.npy) to a watertight mesh using alpha shapes."""
    points = np.load(point_cloud_file)
    vtk_points = vtk.vtkPoints()
    for point in points:
        vtk_points.InsertNextPoint(point)
    poly_data = vtk.vtkPolyData()
    poly_data.SetPoints(vtk_points)
    delaunay = vtk.vtkDelaunay3D()
    delaunay.SetInputData(poly_data)
    delaunay.SetAlpha(alpha)
    delaunay.Update()
    surface_filter = vtk.vtkDataSetSurfaceFilter()
    surface_filter.SetInputConnection(delaunay.GetOutputPort())
    surface_filter.Update()
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(output_mesh_file)
    writer.SetInputConnection(surface_filter.GetOutputPort())
    writer.Write()
    print(f"Watertight mesh saved to: {output_mesh_file}")

def mesh_to_volume(input_mesh_file, volume_file, spacing=(1.0, 1.0, 1.0)):
    """Converts a watertight mesh to a volumetric representation (e.g., voxel grid)."""
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(input_mesh_file)
    reader.Update()
    bounds = reader.GetOutput().GetBounds()
    image_data = vtk.vtkImageData()
    image_data.SetSpacing(spacing)
    dims = [
        int((bounds[1] - bounds[0]) / spacing[0]),
        int((bounds[3] - bounds[2]) / spacing[1]),
        int((bounds[5] - bounds[4]) / spacing[2]),
    ]
    image_data.SetDimensions(dims)
    image_data.SetOrigin(bounds[0], bounds[2], bounds[4])
    stencil = vtk.vtkPolyDataToImageStencil()
    stencil.SetInputConnection(reader.GetOutputPort())
    stencil.SetOutputSpacing(spacing)
    stencil.SetOutputOrigin(bounds[0], bounds[2], bounds[4])
    stencil.SetOutputWholeExtent(image_data.GetExtent())
    stencil.Update()
    volume = vtk.vtkImageStencil()
    volume.SetInputData(image_data)
    volume.SetStencilConnection(stencil.GetOutputPort())
    volume.ReverseStencilOff()
    volume.SetBackgroundValue(0)
    volume.Update()
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(volume_file)
    writer.SetInputConnection(volume.GetOutputPort())
    writer.Write()
    print(f"Volume saved to: {volume_file}")

def clean_and_connect_mesh(input_mesh):
    """Cleans the mesh and extracts the largest connected component."""
    connectivity_filter = vtk.vtkConnectivityFilter()
    connectivity_filter.SetInputData(input_mesh)
    connectivity_filter.SetExtractionModeToLargestRegion()
    connectivity_filter.Update()
    return connectivity_filter.GetOutput()

def smooth_mesh(input_mesh, iterations=30, relaxation_factor=0.1):
    """Smooths the mesh to improve quality."""
    smoother = vtk.vtkSmoothPolyDataFilter()
    smoother.SetInputData(input_mesh)
    smoother.SetNumberOfIterations(iterations)
    smoother.SetRelaxationFactor(relaxation_factor)
    smoother.FeatureEdgeSmoothingOff()
    smoother.BoundarySmoothingOn()
    smoother.Update()
    return smoother.GetOutput()

def generate_voronoi_from_mesh(mesh_file, voronoi_output_file):
    """
    Generates a Voronoi diagram based on the points of a watertight mesh.
    """
    # Read the watertight mesh
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(mesh_file)
    reader.Update()
    mesh = reader.GetOutput()

    # Extract points from the mesh
    points = mesh.GetPoints()

    # Create a polydata object to store the points
    point_polydata = vtk.vtkPolyData()
    point_polydata.SetPoints(points)

    # Generate the Delaunay tessellation
    delaunay_filter = vtk.vtkDelaunay3D()
    delaunay_filter.SetInputData(point_polydata)
    delaunay_filter.Update()

    # Write the Delaunay tessellation to a file (as a substitute for Voronoi)
    writer = vtk.vtkXMLUnstructuredGridWriter()
    writer.SetFileName(voronoi_output_file)
    writer.SetInputData(delaunay_filter.GetOutput())
    writer.Write()

    print(f"Voronoi diagram saved to: {voronoi_output_file}")

def extract_centerlines_from_voronoi(voronoi_file, centerline_output_file):
    """
    Extracts the centerlines from a Voronoi diagram.
    """
    # Read the Voronoi diagram
    reader = vtk.vtkXMLUnstructuredGridReader()
    reader.SetFileName(voronoi_file)
    reader.Update()
    voronoi = reader.GetOutput()

    # Extract the edges of the Voronoi diagram
    geometry_filter = vtk.vtkGeometryFilter()
    geometry_filter.SetInputData(voronoi)
    geometry_filter.Update()
    voronoi_edges = geometry_filter.GetOutput()

    # Compute the bounds of the Voronoi edges
    bounds = voronoi_edges.GetBounds()
    inlet_point = [(bounds[0] + bounds[1]) / 2, (bounds[2] + bounds[3]) / 2, bounds[4]]  # Bottom center
    outlet_point = [(bounds[0] + bounds[1]) / 2, (bounds[2] + bounds[3]) / 2, bounds[5]]  # Top center

    # Create seed points for inlet and outlet
    seed_points = vtk.vtkPoints()
    seed_points.InsertNextPoint(inlet_point)
    seed_points.InsertNextPoint(outlet_point)

    # Create a vtkPolyData object to store the seed points
    seed_polydata = vtk.vtkPolyData()
    seed_polydata.SetPoints(seed_points)

    # Use vmtkCenterlines to extract the centerlines
    centerline_filter = vmtkCenterlines()
    centerline_filter.Surface = voronoi_edges
    centerline_filter.SeedSelectorName = 'pointlist'

    # Initialize the SeedSelector and set the seed points
    seed_selector = vmtkPointListSeedSelector()
    seed_selector.SeedPoints = seed_polydata  # Set the seed points directly
    centerline_filter.SeedSelector = seed_selector

    # Execute the centerline extraction
    centerline_filter.Execute()

    # Extract the centerline as a polydata
    centerline_polydata = centerline_filter.Centerlines

    # Write the centerline to a file
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(centerline_output_file)
    writer.SetInputData(centerline_polydata)
    writer.Write()

    print(f"Centerlines saved to: {centerline_output_file}")

def visualize_mesh_with_centerline(mesh_file, centerline_file):
    """
    Visualizes the mesh and its centerline together using VTK.
    """
    # Read the mesh file
    mesh_reader = vtk.vtkXMLPolyDataReader()
    mesh_reader.SetFileName(mesh_file)
    mesh_reader.Update()
    mesh = mesh_reader.GetOutput()

    # Read the centerline file
    centerline_reader = vtk.vtkXMLPolyDataReader()
    centerline_reader.SetFileName(centerline_file)
    centerline_reader.Update()
    centerline = centerline_reader.GetOutput()

    # Create a mapper and actor for the mesh
    mesh_mapper = vtk.vtkPolyDataMapper()
    mesh_mapper.SetInputData(mesh)
    mesh_actor = vtk.vtkActor()
    mesh_actor.SetMapper(mesh_mapper)
    mesh_actor.GetProperty().SetColor(0.8, 0.8, 0.8)  # Light gray

    # Create a mapper and actor for the centerline
    centerline_mapper = vtk.vtkPolyDataMapper()
    centerline_mapper.SetInputData(centerline)
    centerline_actor = vtk.vtkActor()
    centerline_actor.SetMapper(centerline_mapper)
    centerline_actor.GetProperty().SetColor(1.0, 0.0, 0.0)  # Red
    centerline_actor.GetProperty().SetLineWidth(3)

    # Create a renderer, render window, and interactor
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    # Add the actors to the renderer
    renderer.AddActor(mesh_actor)
    renderer.AddActor(centerline_actor)
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background

    # Render and start interaction
    render_window.Render()
    render_window_interactor.Start()

def slice_mesh_and_connect_centroids(mesh_file, num_slices=10):
    """
    Slices the mesh into a specified number of slices along the Z-axis and connects the centroids of each slice.
    """
    # Read the mesh file
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(mesh_file)
    reader.Update()
    mesh = reader.GetOutput()

    # Get the bounds of the mesh
    bounds = mesh.GetBounds()
    z_min, z_max = bounds[4], bounds[5]
    slice_height = (z_max - z_min) / num_slices

    # Initialize a list to store centroids
    centroids = []

    # Slice the mesh and compute centroids
    for i in range(num_slices):
        z_lower = z_min + i * slice_height
        z_upper = z_lower + slice_height

        # Create a plane to slice the mesh
        plane = vtk.vtkPlane()
        plane.SetOrigin(0, 0, z_lower)
        plane.SetNormal(0, 0, 1)

        # Clip the mesh with the plane
        clipper = vtk.vtkClipPolyData()
        clipper.SetInputData(mesh)
        clipper.SetClipFunction(plane)
        clipper.Update()
        sliced_mesh = clipper.GetOutput()

        # Compute the centroid of the sliced mesh
        center_of_mass_filter = vtk.vtkCenterOfMass()
        center_of_mass_filter.SetInputData(sliced_mesh)
        center_of_mass_filter.Update()
        centroid = center_of_mass_filter.GetCenter()
        centroids.append(centroid)

    # Create a polyline to connect the centroids
    points = vtk.vtkPoints()
    polyline = vtk.vtkPolyLine()
    polyline.GetPointIds().SetNumberOfIds(len(centroids))

    for i, centroid in enumerate(centroids):
        points.InsertNextPoint(centroid)
        polyline.GetPointIds().SetId(i, i)

    # Create a polydata to store the polyline
    polyline_data = vtk.vtkPolyData()
    polyline_data.SetPoints(points)
    cells = vtk.vtkCellArray()
    cells.InsertNextCell(polyline)
    polyline_data.SetLines(cells)

    # Visualize the polyline
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(polyline_data)
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(1.0, 0.0, 0.0)  # Red

    # Create a renderer, render window, and interactor
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    # Add the actor to the renderer
    renderer.AddActor(actor)
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background

    # Render and start interaction
    render_window.Render()
    render_window_interactor.Start()

def point_cloud_to_vtp(point_cloud_file, output_vtp_file):
    """
    Converts a point cloud (.npy) to a .vtp file for visualization in VTK.
    """
    # Load the point cloud from the .npy file
    points = np.load(point_cloud_file)

    # Create a vtkPoints object and add the points to it
    vtk_points = vtk.vtkPoints()
    for point in points:
        vtk_points.InsertNextPoint(point)

    # Create a vtkPolyData object to store the points
    poly_data = vtk.vtkPolyData()
    poly_data.SetPoints(vtk_points)

    # Write the vtkPolyData to a .vtp file
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(output_vtp_file)
    writer.SetInputData(poly_data)
    writer.Write()

    print(f"Point cloud saved to: {output_vtp_file}")

# Convert Point Cloud to Watertight Mesh
Run the `point_cloud_to_watertight_mesh` function with the specified input and output file paths.

In [33]:
# Input and output file paths
point_cloud_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\accumulated_point_cloud_frame_100.npy"
output_mesh_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\output_mesh.vtp"

# Convert point cloud to watertight mesh
point_cloud_to_watertight_mesh(point_cloud_file, output_mesh_file, alpha=0.1)

Watertight mesh saved to: C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\output_mesh.vtp


# Generate Voronoi
Run the `generate_voronoi_from_mesh` function to extract the voronoi from the watertight mesh. 

In [34]:
# File paths
mesh_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\output_mesh.vtp"
voronoi_output_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\voronoi_output.vtu"

# Generate Voronoi diagram from watertight mesh
generate_voronoi_from_mesh(mesh_file, voronoi_output_file)

Voronoi diagram saved to: C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\voronoi_output.vtu


# Extract Centerline
Run the `extract_centerline_from_voronoi` function to extract the centerline from the voronoi.

In [35]:
# Input and output file paths
voronoi_output_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\voronoi_output.vtu"
centerline_output_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\centerline_from_voronoi.vtp"

# Extract centerlines from Voronoi diagram
extract_centerlines_from_voronoi(voronoi_output_file, centerline_output_file)

Cleaning surface.
Triangulating surface.
vmtkPointListSeedSelector Error: SourcePoints not set.


SystemExit: 

# Visualize Mesh and Centerline
Run the `visualize_mesh_with_centerline` function to visualize the mesh and its centerline together.

In [ ]:
# Visualize the mesh and centerline together
visualize_mesh_with_centerline(output_mesh_file, centerline_file)

# Slice Mesh and Connect Centroids
Run the `slice_mesh_and_connect_centroids` function to slice the mesh and connect centroids with a line.

In [ ]:
# Slice the mesh and connect centroids
slice_mesh_and_connect_centroids(output_mesh_file, num_slices=10)

# Convert Point Cloud to VTP
Run the `point_cloud_to_vtp` function to convert the point cloud to a `.vtp` file.

In [ ]:
# Convert the point cloud to .vtp
point_cloud_to_vtp(point_cloud_file, output_mesh_file)

Point cloud saved to: C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\output_mesh.vtp
